# Asteroid Belt
Make sim of asteroid belt.

In [122]:
import PIL # reminder that this is a requirement
from scipy.misc import imread
import matplotlib.pyplot as plt

def write_png(params):
    sim = rebound.Simulation.from_file(params['filename'])
    ps = sim.particles
    plotparticles = params['plotparticles']       # particles that should be displayed
    plottransits = params['plottransits']         # whether we should highlight transits
    color = params['color']                       # colors for each of the particles (including the star)
    figscale= params['figscale']
    
    #only plot particles once they have transisited for the first time
    '''
    plotparticles=[]
    show_orbit = False
    for i in range(len(transits.events)):
        if sim.t > transits.events[i]['time'] and transits.events[i]['target'] not in plotparticles:
            plotparticles.append(transits.events[i]['target'])
    plotparticles=sorted(plotparticles)
    if len(plotparticles)>0:
        show_orbit = True
    '''
    show_orbit = True
    
    # First make a color list that only includes the particles we want to plot
    #coloriterator = [color[i] for i in plotparticles]
    lw=1 # linewidth

    fig = rebound.OrbitPlot(sim, figsize=(8,8), show_orbit=show_orbit, color=False, lw=lw, plotparticles=plotparticles,lim=figscale)


    ax = fig.axes[0] # turn off axes
    ax.axis('off')
    
    if show_orbit:
        #refsize=25*lw # this is what REBOUND uses for size of circles in call to plt.scatter
        refsize=35*lw 
        for i in plotparticles:
            ax.scatter(ps[i].x, ps[i].y, s=refsize, color='black', marker='o', zorder=4)
            
    #modify to enlarge planet on its entrance
    if plottransits:
        refsize=35*lw 
        for i in plotparticles: # overplot markers so planets have their own color
            #ax.scatter(ps[i].x, ps[i].y, s=refsize, color=color[i], marker='o', zorder=4)
            scale=ps[i].a/3     # length scale for making dots bigger
            size=refsize
            if ps[i].x > 0 and np.abs(ps[i].y)/scale < 1: # increase size when particle is within `scale` of y=0, and on right
                size *= 1+8*np.exp(-np.abs(ps[i].y)/scale)
                ax.scatter(ps[i].x, ps[i].y, s=size, color='black', marker='o', zorder=5)

    fig.savefig('tmp/pngs/{0:0=5d}.png'.format(params['frame_ctr']), transparent=False, dpi=300)
    plt.close(fig)  

Now we make our movie. We begin by importing what we need, and load our REBOUND binary simulation of the TRAPPIST-1 system that we want to animate/sonify. For convenience, we redefine the time in the simulation to 0, and rescale time so that every orbit of the innermost planet corresponds to one unit of time in the simulation (which we define as one beat in the music).

In [135]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound

sim0 = rebound.Simulation.from_file("../binaries/solarsystemI.bin")
P_mars=sim0.particles[-1].P

#filename = "../binaries/asteroidbelt.bin"
filename = "../binaries/asteroidbelt_m.bin" #includes mars as planet 1

sim = rebound.Simulation.from_file(filename)
sim.t = 0
#ss.rescale_time(sim, sim.particles[-1].P) #set time to period of outer asteroid
#ss.rescale_time(sim, P_mars) #set time to period of mars, why doesn't this work? doesn't record events
ss.rescale_time(sim, sim.particles[1].P) #set time to period of mars
sim.dt=sim.particles[1].P/1000.
print(sim.particles[1].P,sim.dt)

1.0000000000000007 0.0010000000000000007


We chose to begin the movie at 60 beats per minute. We have to convert this to simulation time per second. Above we rescaled time so that one orbit of the outer planet is one unit of simulation time, so 60 bpm / 60 = 1 beats per second. We also assign colors to each of the particles in the simulation.

In [136]:
bpm = 34. 
frames = ss.FrameRecorder(sim, time_per_sec=bpm/60)
transits = ss.EventRecorder(sim, lambda sim, i: sim.particles[i].y)

# The commented out colors are the colors in the video, but we invert the colors in the post-processing, so we use
# the corresponding inverse colors instead
# actualcolors = ['None','grey','darkorange','forestgreen','cyan','deepskyblue','violet','darkviolet']
frames.color = [None, 'grey','royalblue','darkorange','cyan'] #not suing
frames.figscale = 1.1*sim.particles[-1].a

We also start our MIDI file now, so that we can enter the right tempo information as we go. We will add all the notes at the end:

In [137]:
from midiutil import MIDIFile

midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=sim.t, tempo=bpm) 

We now choreograph the movie. We begin by staggering in the planets from the outside in, one bar (4 beats) at a time. We want to show and play the planets' transits, so we set `frames.plotparticles` and `transits.targets` respectively. 

# Test

May need to integrate backwards first to have asteroids orbiting before planets enter

In [140]:
frames.plottransits = False
planets = list(range(1,sim.N))
ps=planets
frames.plotparticles = ps
transits.targets = []
#sim.integrate(sim.t-8) #back up, show mars to scale size
#frames.plotparticles = ps[1:] #don't show mars
#sim.integrate(sim.t+8)
for i in range(2):
    print("t = {0}".format(sim.t))
    sim.integrate(sim.t+4)   

0.9999999999999996 0.015707311908910296
t = 16.0
t = 20.0
1.0000000000000002 0.015458302635266962


In [141]:
print(frames.events)

[{'time': 0.01888890075683595, 'target': None, 'fps': 30, 'time_per_sec': 0.5666666666666667, 'frame_ctr': 0, 'elapsed_time': 0.03705882352941179, 'plotparticles': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189

# MIDI File

We now write the MIDI file. We scale all transit notes to the outermost planet, which we assign to a C4 note (MIDI note 48). 

In [57]:
transit_notes = ss.calc_midi_notes(sim.particles, ref_note=48, ref_ID=-1)

for transit in transits.events:
    midifile.addNote(track=0, channel=transit['target'], pitch=transit_notes[transit['target']], time=transit['time'], duration=1, volume=100)

with open("./asteroidbelt.mid", "wb") as f:
    midifile.writeFile(f)

For the actual movie, we loaded the MIDI in Logic (music mixing software), and assigned each of these conjunctions to a different drum.

# Write images to temporary folder for movie

Now we write the images. This is parallelized, but making matplotlib plots is slow, so this can take a couple hours! (the movie is 2 mins long).

In [82]:
%%time
from subprocess import call
call("rm -f tmp/pngs/*", shell=True)
pool = rebound.InterruptiblePool()
res = pool.map(write_png, frames.events)

CPU times: user 15 ms, sys: 23.3 ms, total: 38.2 ms
Wall time: 3.97 s


# Write movie file

You now have all the frames for the movie in `systemsounds/jupyter_examples/tmp/pngs`, as well as the MIDI file `systemsounds/jupyter_examples/trappist.mid`, and can stitch them together into a movie using your favorite software. For some ideas and options, see [MovieEditingSoftware.ipynb](MovieEditingSoftware.ipynb)


In [66]:
moviename = 'asteroidbelt.mp4'
fps = 30
try:
    call("rm -f {0}".format(moviename), shell=True)
except:
    pass
call("ffmpeg -r {0} -i tmp/pngs/%05d.png -c:v libx264 -pix_fmt yuv420p {1}".format(fps, moviename), shell=True)
#call("open test.mp4", shell=True)

0